[View in Colaboratory](https://colab.research.google.com/github/nikhilsingh291/eip_theinkers/blob/master/NIKHIL_SINGH_Session7.ipynb)

#Deep Learning for Visual Question Answering


![alt text](https://avisingh599.github.io/images/vqa/sample_results.jpg)

This is about answering general english questions based on images by deep learning.

I will be using LSTM+CNN approach to solve this problem and get a better accuracy.

In [4]:
!mkdir data
%cd data


/content/data


In [5]:
# Downloads the training and validation sets from visualqa.org. 

!wget http://visualqa.org/data/mscoco/vqa/Questions_Train_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Questions_Val_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Annotations_Train_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Annotations_Val_mscoco.zip

!unzip \*.zip

--2018-06-03 16:39:38--  http://visualqa.org/data/mscoco/vqa/Questions_Train_mscoco.zip
Resolving visualqa.org (visualqa.org)... 128.173.88.40
Connecting to visualqa.org (visualqa.org)|128.173.88.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21985607 (21M) [application/zip]
Saving to: ‘Questions_Train_mscoco.zip’

Questions_Train_msc 100%[===================>]  20.97M  10.7MB/s    in 2.0s    

2018-06-03 16:39:40 (10.7 MB/s) - ‘Questions_Train_mscoco.zip’ saved [21985607/21985607]

--2018-06-03 16:39:41--  http://visualqa.org/data/mscoco/vqa/Questions_Val_mscoco.zip
Resolving visualqa.org (visualqa.org)... 128.173.88.40
Connecting to visualqa.org (visualqa.org)|128.173.88.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10594497 (10M) [application/zip]
Saving to: ‘Questions_Val_mscoco.zip’

Questions_Val_mscoc 100%[===================>]  10.10M  5.99MB/s    in 1.7s    

2018-06-03 16:39:43 (5.99 MB/s) - ‘Questions_Val_mscoco.zip’

In [6]:
!ls
!cd ..
!mkdir features

Annotations_Train_mscoco.zip
Annotations_Val_mscoco.zip
mscoco_train2014_annotations.json
mscoco_val2014_annotations.json
MultipleChoice_mscoco_train2014_questions.json
MultipleChoice_mscoco_val2014_questions.json
OpenEnded_mscoco_train2014_questions.json
OpenEnded_mscoco_val2014_questions.json
Questions_Train_mscoco.zip
Questions_Val_mscoco.zip


In [7]:
%cd features
!ls
# Downloads and unzips the VGG features computed on the COCO dataset. 

!wget http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
!unzip coco.zip -d .

/content/data/features
--2018-06-03 16:40:01--  http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip [following]
--2018-06-03 16:40:01--  https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764830639 (729M) [application/zip]
Saving to: ‘coco.zip’

coco.zip             53%[=========>          ] 390.45M  13.9MB/s    eta 23s    

coco.zip            100%[===================>] 729.40M  15.9MB/s    in 48s     

2018-06-03 16:40:49 (15.2 MB/s) - ‘coco.zip’ saved [764830639/764830639]

Archive:  coco.zip
  inflating: ./coco/dataset.json     
  inflating: ./coco/readme.txt       
  inflating: ./coco/vgg_feats.mat    


In [8]:
!ls

coco  coco.zip


In [14]:
!pwd
!ls
!pip install spacy

/content/data/features
coco  coco.zip
  Using cached https://files.pythonhosted.org/packages/3c/31/e60f88751e48851b002f78a35221d12300783d5a43d4ef12fbf10cca96c3/spacy-2.0.11.tar.gz
  Using cached https://files.pythonhosted.org/packages/5e/31/c8c1ecafa44db30579c8c457ac7a0f819e8b1dbc3e58308394fff5ff9ba7/murmurhash-0.28.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/f8/9e/273fbea507de99166c11cd0cb3fde1ac01b5bc724d9a407a2f927ede91a1/cymem-1.31.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/1b/ac/7c17b1fd54b60972785b646d37da2826311cca70842c011c4ff84fbe95e0/preshed-1.0.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/55/fd/e9f36081e6f53699943381858848f3b4d759e0dd03c43b98807dde34c252/thinc-6.10.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/9e/9b/62c60d2f5bc135d2aa1d8c8a86aaf84edb719a59c7f11a4316259e61a298/plac-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/aa/9b065a76b9af472437a0059f77e8f96

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thinc ... - \ | / - \ | / - \ | / -

 / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python


In [17]:
!python3 -m spacy download en

    100% |████████████████████████████████| 37.4MB 71.4MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [23]:
!pip install utils #grouper

In [24]:
import numpy as np
import scipy.io
import sys

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Reshape
from keras.layers import Merge
from keras.layers.recurrent import LSTM
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, RemoteMonitor

from sklearn.externals import joblib
from sklearn import preprocessing

#from spacy.en import English

from utils import grouper, selectFrequentAnswers
from features import get_images_matrix, get_answers_matrix, get_questions_tensor_timeseries

ImportError: ignored

In [27]:
word_vec_dim= 300
img_dim = 4096
max_len = 30
nb_classes = 1000

questions_train = open('../data/preprocessed/questions_train2014.txt', 'r').read().decode('utf8').splitlines()
questions_lengths_train = open('../data/preprocessed/questions_lengths_train2014.txt', 'r').read().decode('utf8').splitlines()
answers_train = open('../data/preprocessed/answers_train2014_modal.txt', 'r').read().decode('utf8').splitlines()
images_train = open('../data/preprocessed/images_train2014.txt', 'r').read().decode('utf8').splitlines()
vgg_model_path = '../features/coco/vgg_feats.mat'

FileNotFoundError: ignored